In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

#hyper parameters
block_size=8
batch_size=4

cuda


In [4]:
with open("wizard_of_oz.txt", 'r', encoding="utf-8") as f:
    text=f.read()
# creating an array of all the characters in text file
chars=sorted(set(text))   
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


# tokenizing the character

In [5]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda l:"".join([int_to_string[i] for i in l])

In [6]:
#to encode a word we use the following 
print(encode("hellow"))
encoded_word=encode("hellow")
decode_word=decode(encoded_word)
print(f"the encoded word is: {encoded_word}")
print(f"the decoded word is: {decode_word}")

[61, 58, 65, 65, 68, 76]
the encoded word is: [61, 58, 65, 65, 68, 76]
the decoded word is: hellow


In [7]:
# encoding text data to a tourch tensor
data=torch.tensor(encode(text), dtype=torch.long)
print(data[:50])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25])


In [13]:
#spliting data into train and val set
n=int(0.8 * len(data))

def get_batch(split):
    data=train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y=get_batch("train")
print(f"input: {x} ... output: \n {y}")

train_data=data[:n]
val_data=data[n:]

tensor([171425, 107385, 174869, 141494])
input: tensor([[ 1, 44, 61, 58,  1, 60, 71, 58],
        [ 0,  0,  0, 44, 61, 58,  1, 66],
        [79,  1, 54, 59, 73, 58, 71,  1],
        [57,  1, 62, 67, 73, 68,  1, 73]], device='cuda:0') ... output: 
 tensor([[44, 61, 58,  1, 60, 71, 58],
        [ 0,  0, 44, 61, 58,  1, 66],
        [ 1, 54, 59, 73, 58, 71,  1],
        [ 1, 62, 67, 73, 68,  1, 73]], device='cuda:0')


In [9]:
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when the input is: {context} target is {target}")

when the input is: tensor([80]) target is 28
when the input is: tensor([80, 28]) target is 39
when the input is: tensor([80, 28, 39]) target is 42
when the input is: tensor([80, 28, 39, 42]) target is 39
when the input is: tensor([80, 28, 39, 42, 39]) target is 44
when the input is: tensor([80, 28, 39, 42, 39, 44]) target is 32
when the input is: tensor([80, 28, 39, 42, 39, 44, 32]) target is 49
when the input is: tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is 1


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeddeing_table=nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets):
        logits = self.token_embeddeing_table(index)
        #we unpack the dimension using the .shape
        batch, time, channel=logits.shape
        #we pack the dimension using the .view  
        logits=logits.view(batch*time, channel)
        targets=targets.view(batch*time)
        loss=F.cross_entropy(logits, targets)
        return logits